In [ ]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms, datasets
import torchvision.models as models
import matplotlib.pyplot as plt
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm, trange
import math

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/OriginHealth/Task1/training_images/trainimages.zip -d /content

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data_path, csv_file=None, transform=None):
      self.annotations = pd.read_csv(csv_file)
      self.data_path = data_path
      self.transform = transform

    def __len__(self):
      return len(self.annotations) 

    def __getitem__(self, index):
      for name in os.listdir(self.data_path):
        path = os.path.join(self.data_path, self.annotations.iloc[index,0]) + '.png'
        img = Image.open(path)
        img = transforms.ToTensor()(img)
        label=torch.tensor(int(self.annotations.iloc[index,1]))

        if self.transform:
          img = self.transform(img)
        return img, label

In [ ]:
transform  = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Normalize(mean=[0.5], std=[0.5]),
    transforms.RandomHorizontalFlip(p=0.8)
])

In [ ]:
data_path = '/content/images'
csv_file='/content/drive/MyDrive/OriginHealth/Task1/training_image_labels/training_image_labels.csv'

dataset = CustomDataset(data_path, csv_file, transform=transform)

In [ ]:
train_dataset, val_dataset = random_split(dataset, [1481, 165])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [ ]:
#one hot encoding the labels
num_classes = 4

for imgs, labels in train_loader:
    print("Batch of images has shape: ",imgs.shape)
    one_hot_labels = F.one_hot(labels, num_classes=num_classes)
    print("Batch of one hot encoded labels: \n", one_hot_labels)



In [ ]:
# pre-trained VGG16 model
vgg16 = models.vgg16(pretrained=True)

# Adapt the first convolutional layer to work on 1 channel input
new_first_conv = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1)
with torch.no_grad():
    new_first_conv.weight[:, :3, :, :] = torch.mean(vgg16.features[0].weight, dim=1, keepdim=True)
    new_first_conv.weight[:, 3:, :, :] = 0
    new_first_conv.bias = vgg16.features[0].bias

# Replace the first convolutional layer in the VGG16 model
vgg16.features[0] = new_first_conv

# Replace fully connected layers with new layers
vgg16.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 4096),  
    nn.ReLU(inplace=True),         
    nn.Linear(4096, 512),
    nn.ReLU(inplace=True),
    nn.Dropout(),
    nn.Linear(512, 10),  
    nn.ReLU(inplace=True),
    nn.Linear(10, 4)  
)



for param in vgg16.classifier.parameters():
    param.requires_grad = True

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [ ]:
# Freeze all layers convolution layers
for param in vgg16.parameters():
    param.requires_grad = False
for param in vgg16.classifier[-6:].parameters():
    param.requires_grad = True

In [ ]:
lce=nn.CrossEntropyLoss()

optimizer = optim.Adam(vgg16.parameters(), lr=0.001)


scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
vgg16.to(device)
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
num_epochs=10
# Train the model
for epoch in trange(num_epochs):
    # Set the model to train mode
    vgg16.train()

    num_tcorrect = 0
    num_tsamples = 0
    
    for inputs, labels in train_loader:
      
      inputs, labels = inputs.to(device), labels.to(device)

      
      optimizer.zero_grad()
        
      
      outputs = vgg16(inputs)
      loss = lce(outputs, labels)
        
      
      loss.backward()
      optimizer.step()


      _, preds = torch.max(outputs, 1)
      num_tcorrect += (preds == labels).sum().item()
      num_tsamples += inputs.size(0)
    
    train_acc= num_tcorrect/num_tsamples
    
    # Set the model to evaluation mode
    vgg16.eval()
    
    
    with torch.no_grad():
        val_loss = 0
        num_correct = 0
        num_samples = 0
        for inputs, labels in val_loader:
          
          inputs, labels = inputs.to(device), labels.to(device)

          outputs = vgg16(inputs)
          val_loss += lce(outputs, labels).item() * inputs.size(0)
          _, preds = torch.max(outputs, 1)
          num_correct += (preds == labels).sum().item()
          num_samples += inputs.size(0)
        val_loss /= len(val_dataset)
        val_acc = num_correct / num_samples
    
 
    print(f'Epoch {epoch+1}/{num_epochs} - Training loss: {loss:.4f} - Validation loss: {val_loss:.4f} - Validation accuracy: {val_acc:.4f} - Training accuracy: {train_acc:.4f}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1/10 - Training loss: 0.9755 - Validation loss: 1.0155 - Validation accuracy: 0.6727 - Training accuracy: 0.4990
Epoch 2/10 - Training loss: 0.8341 - Validation loss: 0.8980 - Validation accuracy: 0.7697 - Training accuracy: 0.6698
Epoch 3/10 - Training loss: 0.5533 - Validation loss: 0.8095 - Validation accuracy: 0.7576 - Training accuracy: 0.7934
Epoch 4/10 - Training loss: 0.3504 - Validation loss: 0.7313 - Validation accuracy: 0.8121 - Training accuracy: 0.8454
Epoch 5/10 - Training loss: 0.1560 - Validation loss: 0.6072 - Validation accuracy: 0.8485 - Training accuracy: 0.8771
Epoch 6/10 - Training loss: 0.2002 - Validation loss: 0.5701 - Validation accuracy: 0.8485 - Training accuracy: 0.9001
Epoch 7/10 - Training loss: 0.0720 - Validation loss: 0.5895 - Validation accuracy: 0.8545 - Training accuracy: 0.9156
Epoch 8/10 - Training loss: 0.1866 - Validation loss: 0.5163 - Validation accuracy: 0.8909 - Training accuracy: 0.9237
Epoch 9/10 - Training loss: 0.0956 - Validation 

In [ ]:
torch.save(vgg16.state_dict(), '/content/drive/MyDrive/OriginHealth/Task1/model_vgg16.pt')

TESTING

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_path, transform=None):
      self.data_path = data_path
      self.transform = transform

    def __len__(self):
      return len(os.listdir(self.data_path)) 

    def __getitem__(self, index):
      for name in os.listdir(self.data_path):
        path = os.path.join(self.data_path, name)
        img = Image.open(path)
        img = self.transform(img)
        
        return img

In [ ]:
test_transform  = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
])

In [ ]:
test_dataset = TestDataset('/content/drive/MyDrive/OriginHealth/Task1/test_images/External_Test_images', transform=test_transform)

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
for imgs in test_loader:
    print("Batch of images has shape: ",imgs.shape)

Batch of images has shape:  torch.Size([32, 1, 224, 224])
Batch of images has shape:  torch.Size([8, 1, 224, 224])


In [ ]:
# Load the saved model state from a file
model_state_dict = torch.load('/content/drive/MyDrive/OriginHealth/Task1/model_vgg16.pt')

# Load the saved model state into the model
vgg16.load_state_dict(model_state_dict)

# Put the model in evaluation mode
vgg16.eval()

VGG(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [ ]:
predictions = []
with torch.no_grad():
  for data in test_loader:
    data = data.to(device)
    output = vgg16(data)
    predictions += list(torch.argmax(output, dim=1))

In [ ]:
label_dictionary={
  '0': "Fetal_brain",
  '1': "Fetal_femur",
  '2': "Fetal_thorax",
  '3': "Fetal_abdomen"
  }

In [ ]:
predictions_labels=[]
for item in predictions:
  print(item)

tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')
t